In [1]:
from elasticsearch import Elasticsearch
es = Elasticsearch()
from elasticsearch_dsl import Search

In [3]:
import requests
res = requests.get('http://localhost:9200')
print(res.content.decode("utf-8"))

{
  "name" : "ASUS",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "iIqVOAu8Q3GC415I7jGCbg",
  "version" : {
    "number" : "6.4.2",
    "build_flavor" : "unknown",
    "build_type" : "unknown",
    "build_hash" : "04711c2",
    "build_date" : "2018-09-26T13:34:09.098244Z",
    "build_snapshot" : false,
    "lucene_version" : "7.4.0",
    "minimum_wire_compatibility_version" : "5.6.0",
    "minimum_index_compatibility_version" : "5.0.0"
  },
  "tagline" : "You Know, for Search"
}



In [ ]:
tables = ['business','checkin','photo','review','tip','user']
for table in tqdm(tables):
    data = []
    _ = es.indices.create(index='yelp_'+table , ignore=400)
    with open('D:\Desktop\CGU\碩班\碩二上/yelp_academic_dataset_{}.json'.format(table), encoding='utf-8') as f:
        i = 0
        for line in f:
            result=es.create(index='yelp_'+table,doc_type=table,id=i,body=json.loads(line))
            i += 1
            if i % 5000 == 0:
                print(i, 'records have been uploaded')
    print(table, 'data finished!!!')

## Elasticsearch DSL to Python DSL (1): term

In [5]:
body = {
    "query":{
        "term":{
            "stars":"5"
        }
    }
}


result = es.search(index="yelp_business",body=body)
result


# 最下方的 'total': 30941筆

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '15249',
    '_index': 'yelp_business',
    '_score': 1.0,
    '_source': {'address': '19400 Jetton Rd, Ste 204',
     'attributes': {'BikeParking': 'True',
      'BusinessAcceptsCreditCards': 'True',
      'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}",
      'Caters': 'False',
      'RestaurantsPriceRange2': '2',
      'RestaurantsTakeOut': 'True',
      'WheelchairAccessible': 'True'},
     'business_id': 'Yh3IX2dg0eLxC9VdTk3gdg',
     'categories': 'Food, Beer, Wine & Spirits',
     'city': 'Cornelius',
     'hours': {'Friday': '11:0-20:0',
      'Monday': '11:0-20:0',
      'Saturday': '11:0-20:0',
      'Sunday': '13:0-18:0',
      'Thursday': '11:0-20:0',
      'Tuesday': '11:0-20:0',
      'Wednesday': '11:0-20:0'},
     'is_open': 1,
     'latitude': 35.476611,
     'longitude': -80.892949,
     'name': 'Winestore',
     'ne

In [6]:
#設定輸出筆數，以確認 python DSL能獲取相同筆數
es.indices.put_settings(index="yelp_business",
                        body= {"index" : {
                                "max_result_window" : 500000
                              }})

{'acknowledged': True}

In [7]:
body1 = Search(using=es , index ='yelp_business').query('term',stars='5').extra(size=100000)
response = body1.execute()

In [8]:
len(response)

30941

## Elasticsearch DSL to Python DSL (2):  aggregation
### 對各星等(stars)的評論數(review_count)進行加總(sum)

In [112]:
body = {
    'aggs': {
        'star':{
            'terms':{
                'field': 'stars'
            },
            'aggs': {
                'review_counts': {
                    'sum': {
                        'field': 'review_count'
                    }
                }
            }
        }
    }
}
result_state_stars = es.search(index="yelp_business",body=body)
result_state_stars

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'aggregations': {'star': {'buckets': [{'doc_count': 35123,
     'key': 4.0,
     'review_counts': {'value': 1878553.0}},
    {'doc_count': 33974, 'key': 3.5, 'review_counts': {'value': 1325853.0}},
    {'doc_count': 30941, 'key': 5.0, 'review_counts': {'value': 380944.0}},
    {'doc_count': 26707, 'key': 4.5, 'review_counts': {'value': 1075874.0}},
    {'doc_count': 24704, 'key': 3.0, 'review_counts': {'value': 735364.0}},
    {'doc_count': 17808, 'key': 2.5, 'review_counts': {'value': 353937.0}},
    {'doc_count': 10266, 'key': 2.0, 'review_counts': {'value': 152658.0}},
    {'doc_count': 4727, 'key': 1.5, 'review_counts': {'value': 68030.0}},
    {'doc_count': 4343, 'key': 1.0, 'review_counts': {'value': 25537.0}}],
   'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0}},
 'hits': {'hits': [{'_id': '15249',
    '_index': 'yelp_business',
    '_score': 1.0,
    '_source': {'address': '19400 Jetton Rd, St

In [98]:
s = Search(using=es , index ='yelp_business')

s.aggs.bucket('star_review', 'terms', field='stars')\
    .metric('review_counts', 'sum', field='review_count')

response1 = s.execute()
# s.to_dict()     #呈現對應至elasticsearch DSL之語法
# print(response1.success())       #若可執行，回報TRUE
print(response1)



<Response: [<Hit(yelp_business/business/15249): {'business_id': 'Yh3IX2dg0eLxC9VdTk3gdg', 'name': 'Winestore...}>, <Hit(yelp_business/business/15252): {'business_id': 'iK3wcWkyJf7nKc3uiyYEBw', 'name': 'Oak Stree...}>, <Hit(yelp_business/business/15260): {'business_id': 'nuiR_GOTnqLEYefq3HCfQA', 'name': 'Steel Val...}>, <Hit(yelp_business/business/15262): {'business_id': 'Py3adiE4LRiTmygj0-XBjg', 'name': 'Classic C...}>, <Hit(yelp_business/business/15270): {'business_id': 'QIQpvNaOLDJYb6KhOMAx0g', 'name': 'Capri Nai...}>, <Hit(yelp_business/business/15272): {'business_id': 'DIvMSVbsrROQPm8kGXxU8Q', 'name': 'Save This...}>, <Hit(yelp_business/business/15222): {'business_id': 'lSSU9OSEeJfFpLUNZnJ8zg', 'name': 'Marion Di...}>, <Hit(yelp_business/business/15207): {'business_id': '-E7QkLxzR1rhR15j3sW9wQ', 'name': 'Maricopa ...}>, <Hit(yelp_business/business/15362): {'business_id': 'aWg_SADy2jOaZEvu09By7w', 'name': 'Gander Mo...}>, <Hit(yelp_business/business/15364): {'business_id': 'NefuhO6Q

In [99]:
#print out 星等與其總評論數
for tag in response1.aggregations.star_review.buckets:
    print(tag.key, tag.review_counts.value)

4.0 1878553.0
3.5 1325853.0
5.0 380944.0
4.5 1075874.0
3.0 735364.0
2.5 353937.0
2.0 152658.0
1.5 68030.0
1.0 25537.0
